In [1]:
import os 
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
load_dotenv()
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
HUGGINGFACEHUB_API_TOKEN=os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
Groq_llm= ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
)

In [4]:
repo_id="Qwen/Qwen3-235B-A22B"
llm=HuggingFaceEndpoint(
    repo_id=repo_id,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
    task="text-generation",
    provider="auto",
    temperature=0.5
    )
chatmodel=ChatHuggingFace(llm=llm)

In [5]:
hf_embeddings = HuggingFaceEndpointEmbeddings(
    model="sentence-transformers/all-mpnet-base-v2",
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

In [6]:
Vector_store=Chroma(
    collection_name="chatbot",
    embedding_function=hf_embeddings
    )

In [7]:
pdf_folder = Path("/Users/sunnykumar/Downloads/AI DEVELOPER – LLM SPECIALIST ASSIGNMENT/pdf")

In [8]:
all_docs = []
for pdf_file in pdf_folder.glob("*.pdf"):
    loader=PyPDFLoader(str(pdf_file))
    docs=loader.load()
    for doc in docs:
        doc.metadata["source"] = pdf_file.name
    all_docs.extend(docs)

In [9]:
all_docs[21].metadata

{'producer': 'Microsoft® Word 2021',
 'creator': 'Microsoft® Word 2021',
 'creationdate': '2025-02-01T03:54:26+05:30',
 'author': 'hss',
 'moddate': '2025-02-01T03:56:01+05:30',
 'title': '',
 'source': 'budget_speech.pdf',
 'total_pages': 60,
 'page': 9,
 'page_label': '10'}

In [10]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=100)
all_splits = text_splitter.split_documents(all_docs)

In [11]:
Vector_store.add_documents(all_splits)

['827bfe26-3bde-4fcf-9cad-f9fef2ec7f13',
 '95604177-014e-413b-8c49-a85daf61fd67',
 '9490b330-85c9-4fec-abd0-d178740f058a',
 '3d021405-1576-4ac3-baad-bf5cbdde2cd2',
 '85b198ba-334b-412c-a5a8-3af2b4ec010d',
 'e3ca02ce-6dd0-400b-ad40-9875cd3d89bb',
 '34195093-a45c-4659-9318-ec145c6a1dca',
 'b0cf087b-7ad5-4a22-a972-7a2440503f81',
 'dceb131b-0709-4ad7-80b0-a6a2c24bd035',
 '1a2a1d98-fb36-4e78-bc9d-516f8e2e8b84',
 'cafca820-6d68-477d-8471-8c887071f05b',
 'bce0de66-f595-4d81-944c-892759f33d11',
 '1d1f715e-2f1d-4979-ac6f-314eb671db82',
 'b063fdb7-6a81-4dee-b4d1-8ce47aea7936',
 '7c5ae48b-9d03-497c-9c25-00d4aaa96d0e',
 'ce66d026-a7b9-45d0-9c68-0f58c8f12b78',
 '6080297d-2bfb-4b0e-8de3-63b24aad2ba4',
 'f4eee255-c606-4019-969b-2daf7cff2628',
 'd36cd1a5-0d7b-4028-a1b6-b934713d7ec2',
 '063a2468-0ffc-445f-85d8-9a1025e11cd8',
 '2a1285d0-c095-4b28-aad5-fef49cf78fcd',
 '0cc3fc97-c113-485b-83d2-3a5ca2ac3703',
 '8fe0fc23-e555-4916-b4d2-e405761dd42b',
 '760b22b2-c5e3-46ad-96ea-db5969d0c776',
 '89aad46e-80fb-

In [12]:
prompt=ChatPromptTemplate.from_template(
    """
    
    Answer the question based on the context provided below. If the answer is not related to context then try to ans the question from your own knowledge , if not possible simply say i dont know.
    <context>
    {context}
    </context>
    Question: {question}
    
    
    """
    )

In [13]:
retriever=Vector_store.as_retriever(search_kwargs={"k": 6})

In [14]:
retriever.get_relevant_documents("budget of india in 2025?")[0].metadata

{'page_label': '5',
 'total_pages': 60,
 'creationdate': '2025-02-01T03:54:26+05:30',
 'page': 4,
 'author': 'hss',
 'producer': 'Microsoft® Word 2021',
 'creator': 'Microsoft® Word 2021',
 'title': '',
 'moddate': '2025-02-01T03:56:01+05:30',
 'source': 'budget_speech.pdf'}

In [15]:
retriever_chain = RetrievalQA.from_chain_type(
    llm=Groq_llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    input_key="question"
)

In [16]:
response=retriever_chain.invoke({'question':"budget of india in 2025?"})
Vector_store.similarity_search("budget of india in 2025?")
response['result']

"The Budget of India for 2025-26, presented by Finance Minister Nirmala Sitharaman, outlines several key financial and policy initiatives:\n\n1. **Fiscal Deficit**: The fiscal deficit is estimated at 4.4% of GDP, slightly lower than the revised estimate of 4.8% for 2024-25.\n\n2. **Total Receipts and Expenditure**: The total receipts (excluding borrowings) are ₹34.96 lakh crore, while the total expenditure is ₹50.65 lakh crore. Net tax receipts are estimated at ₹28.37 lakh crore.\n\n3. **Borrowings**: Net market borrowings from dated securities amount to ₹11.54 lakh crore, with gross market borrowings at ₹14.82 lakh crore.\n\n4. **Policy Initiatives**:\n   - **Asset Monetization Plan 2025-30**: Aims to generate ₹10 lakh crore for new projects.\n   - **Infrastructure Support**: An outlay of ₹1.5 lakh crore for 50-year interest-free loans to states for capital expenditure.\n   - **Jal Jeevan Mission**: Continuation of efforts to provide clean water, with 15 crore households already benef

In [17]:
related_context=retriever.get_relevant_documents("budget of india in 2025?")

In [18]:
for i in related_context:
    print(i.page_content)

Budget 2025-2026 
 
Speech of 
Nirmala Sitharaman 
Minister of Finance 
February 1, 2025 
Hon’ble Speaker,  
 I present the Budget for 2025-26. 
Introduction 
1. This Budget continues our Government’s efforts to: 
a) accelerate growth,  
b) secure inclusive development,  
c) invigorate private sector investments,  
d) uplift household sentiments, and 
e) enhance spending power of India’s rising middle class.  
2. Together, we embark on a journey to unlock our nation’s tremendous 
potential for greater prosperity and global positioning under the leadership of 
Hon’ble Prime Minister Shri Narendra Modi. 
3. As we complete the first quarter of the 21 st century, continuing 
geopolitical headwinds suggest lower  global economic growth over the
19  
 
Budget Estimates 2025-26 
112. Coming to 2025 -26, the total receipts other than  borrowings and the 
total expenditure are estimated at ` 34.96 lakh crore and ` 50.65 lakh crore 
respectively. The net tax receipts are estimated at ` 28.37 lak

In [19]:
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')
COHERE_API_KEY = os.getenv('COHERE_API_KEY')
from langchain_tavily import TavilySearch
from langchain.chat_models import init_chat_model

In [20]:
from langchain_core.messages import ToolMessage

In [21]:
tavily_search_tool = TavilySearch(max_results=5,tavily_api_key=TAVILY_API_KEY,include_answer=True,search_depth="advanced")
all_tools=[tavily_search_tool]

# Initializing cohere LLM with tool call
llm = init_chat_model("command-a-03-2025", model_provider="cohere")
llm_with_tools=llm.bind_tools(all_tools)

In [22]:
result = llm_with_tools.invoke('current date')
# if tool call made by LLM
tool_calls=result.additional_kwargs.get("tool_calls",[])
if tool_calls:
    if tool_calls[0]['function']['name'] == "tavily_search":
        tool_args= tool_calls[0]['function']['arguments']
        tool_result=tavily_search_tool.invoke(tool_args)
        tool_message=ToolMessage(
            content=tool_result['answer'],
            tool_call_id=tool_calls[0]['id'],
        )
        final_result = llm_with_tools.invoke([result,tool_message])
        print(final_result.content)

The current date is August 16, 2025.


In [25]:
COHERE_API_KEY = os.getenv('COHERE_API_KEY')
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')

In [29]:
from pydantic import BaseModel, Field
from typing import Literal


In [38]:
class QuestionClassifierSchema(BaseModel):
    question: Literal["RAG", "Search_node"] = Field(description='Classify the question into RAG or Search_node based on the context and question')

In [39]:
llm = init_chat_model("command-a-03-2025", model_provider="cohere")

In [ ]:
def classify_question(state: ChatState):
    question = state["question"]
    context= state["context"]
    classification_prompt = f"""
    Classify the question into RAG if the question can be answered using the provided context, otherwise classify it into Search_node if the question requires real-time information or general knowledge.
    
    Question: {question}
    Context: {context}
    """
    response = structured_llm_for_classification.invoke(classification_prompt).question
    return {"next": response}

In [40]:
structured_llm=llm.with_structured_output(QuestionClassifierSchema)

In [46]:
structured_llm.invoke(
    "who is sunny ans using RAG?"
).question

'RAG'